In [24]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open("stop_words_en.txt") as f:
    content = f.readlines()

stop_words = [x.strip() for x in content] 

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,total_words,%d" % 1
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,stop_words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [25]:
%%bash
ls /datasets/stop_words_en.txt

/datasets/stop_words_en.txt


In [26]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [27]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [28]:
! hdfs dfs -ls -h /data/wiki/en_articles_part/articles-part

-rwxrwxrwx   1 jovyan supergroup     73.3 M 2017-11-28 21:41 /data/wiki/en_articles_part/articles-part


In [29]:
%%bash

WC_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=0

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming word count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${WC_DIR} &> stopwords.log


In [31]:
%%bash

# print answer
total_words=$(grep 'total_words' stopwords.log | cut -d= -f2)
stop_words=$(grep 'stop_words' stopwords.log | cut -d= -f2)
percent=$(awk "BEGIN { pc=100*${stop_words}/${total_words}; printf \"%.2f\", pc }")
echo $percent

# print log to stderr for grader
cat stopwords.log >&2

38.44


packageJobJar: [/tmp/hadoop-unjar7659308754455584965/] [] /tmp/streamjob3037350715751085202.jar tmpDir=null
18/04/06 06:42:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/06 06:42:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/06 06:42:48 INFO mapred.FileInputFormat: Total input files to process : 1
18/04/06 06:42:48 INFO mapreduce.JobSubmitter: number of splits:2
18/04/06 06:42:48 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1522476365161_0002
18/04/06 06:42:48 INFO impl.YarnClientImpl: Submitted application application_1522476365161_0002
18/04/06 06:42:49 INFO mapreduce.Job: The url to track the job: http://36cf76319fc6:8088/proxy/application_1522476365161_0002/
18/04/06 06:42:49 INFO mapreduce.Job: Running job: job_1522476365161_0002
18/04/06 06:42:57 INFO mapreduce.Job: Job job_1522476365161_0002 running in uber mode : false
18/04/06 06:42:57 INFO mapreduce.Job:  map 0% reduce 0%
18/04/06 06:43:14 INFO mapred